In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot  #for offline plotting
init_notebook_mode(connected=True)  #for offline plotting
import plotly.graph_objs as go
import plotly.plotly as py

import pandas as pd
import squarify

### Treemap Tryout

In [2]:
data = pd.read_csv('productlevel_by_country.csv')

In [3]:
exports_1996_values = data[data['Year']==1996].groupby('sitc_sdesc')['ExportsFASValueBasisYtdDec'].sum().values
exports_1996_index = data[data['Year']==1996].groupby('sitc_sdesc')['ExportsFASValueBasisYtdDec'].sum().index

In [4]:
x = 0.
y = 0.
width = 400.
height = 200.
values = exports_1996_values

In [5]:
normed = squarify.normalize_sizes(values, width, height)
rects = squarify.squarify(normed, x, y, width, height)

In [6]:
color_brewer = ['rgb(141,211,199)','rgb(255,255,179)','rgb(190,186,218)','rgb(251,128,114)','rgb(128,177,211)','rgb(253,180,98)','rgb(179,222,105)','rgb(252,205,229)','rgb(217,217,217)','rgb(188,128,189)']

In [7]:
shapes = []
annotations = []
counter = 0

In [8]:
for r in rects:
    shapes.append( 
        dict(
            type = 'rect', 
            x0 = r['x'], 
            y0 = r['y'], 
            x1 = r['x']+r['dx'], 
            y1 = r['y']+r['dy'],
            line = dict( width = 2 ),
            fillcolor = color_brewer[counter]
        ) 
    )
    annotations.append(
        dict(
            x = r['x']+(r['dx']/2),
            y = r['y']+(r['dy']/2),
            text = exports_1996_index[counter],
            showarrow = False
        )
    )
    counter = counter + 1
    if counter >= len(color_brewer):
        counter = 0

In [9]:
# For hover text
trace0 = go.Scatter(
    x = [ r['x']+(r['dx']/2) for r in rects ], 
    y = [ r['y']+(r['dy']/2) for r in rects ],
    text = [ str(v) for v in values ], 
    mode = 'text',)

In [10]:
layout = dict(
    height=700, 
    width=1000,
    xaxis=dict(showgrid=False,zeroline=False),
    yaxis=dict(showgrid=False,zeroline=False),
    shapes=shapes,
    annotations=annotations,
    hovermode='closest'
)

In [11]:
# With hovertext
figure = dict(data=[trace0], layout=layout)

In [12]:
iplot(figure)

### Dropdown

In [13]:
types = data.sitc_sdesc.unique()

In [14]:
titles = ['Food and Live Animals Export Amount', 'Beverages and Tobacco Export Amount',
       'Crude Materials and Inedible (except Fuels) Export Amount',
       'Mineral Fuels, Lubricants and Related materials Export Amount',
       'Animal and Vegetable Oils, Fats, and Waxes Export Amount',
       'Chemical and Related Products Export Amount',
       'Manufactured Goods classified chiefly by Material Export Amount',
       'Machinery and Transport Equipment Export Amount',
       'Miscellaneous Manufactured Articles Export Amount',
       'Commodities and Transactions not classified elsewhere Export Amount']

In [15]:
button_names = [
        'Food & Live Animals', 
        'Bev & Tobacco',
       'Crude Materials',
       'Mineral Fuels & Lubs',
       'Animal & Vegie Oils',
       'Chemicals',
       'Mfg. Goods',
       'Machinery & Transport',
       'Misc. Mfg. Articles',
       'Unclassified']

In [16]:
Data = []
counter = 0
for item in types:
    graph = go.Bar(
        x=data[data['sitc_sdesc']==item].groupby('Year')['ExportsFASValueBasisYtdDec'].sum().index,
        y=data[data['sitc_sdesc']==item].groupby('Year')['ExportsFASValueBasisYtdDec'].sum().values,
        name = button_names[counter]
    )
    counter += 1
    Data.append(graph)

In [17]:
buttons = []
counter = 0
for item in button_names:
    vis = [False]*10
    vis[counter] = True
    button = dict(label = item,
                  method = 'update',
                  args = [{'visible': vis},
                          {'title': titles[counter]}])
    buttons.append(button)
    counter += 1

In [18]:
updatemenus = list([
    dict(active=-1,
         buttons=buttons,
        direction='down',
        x = 1.2,
        xanchor = 'right',
        y = 1.2,
        yanchor = 'top')
])


In [19]:
layout = dict(title='US Exports Amount by Product Types', 
              showlegend=True,
              updatemenus=updatemenus,
             xaxis={'title': 'Year'},
             yaxis={'title': 'USD'})

In [20]:
fig = dict(data=Data, layout=layout)

In [21]:
iplot(fig, filename='update_dropdown')